In [1]:
from pathlib import Path
from copy import deepcopy
import os
import zarr
import dask.array as da
from functools import partial
import qtpy

import numpy as np
import pandas as pd
import tifffile
from ipywidgets import IntSlider, VBox
import fastplotlib as fpl

from caiman.motion_correction import high_pass_filter_space
from caiman.summary_images import correlation_pnr

os.environ['WAYLAND_DISPLAY'] = ''
os.environ['RUST_LOG'] = 'info'
os.environ['WINIT_UNIX_BACKEND'] = 'x11'

import mesmerize_core as mc
from mesmerize_viz import *

2024-09-12 12:04:52.561733: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-12 12:04:52.561770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-12 12:04:52.562104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-12 12:04:52.564847: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

In [3]:
parent_path = Path().home() / "caiman_data_org"
movie_path = parent_path / 'animal_01' / 'session_01' / 'plane_1.zarr'

batch_path = parent_path / 'batch.pickle'
mc.set_parent_raw_data_path(str(parent_path))

# you could alos load the registration batch and 
# save this patch in a new dataframe (saved to disk automatically)
try:
    df = mc.load_batch(batch_path)
except (IsADirectoryError, FileNotFoundError):
    df = mc.create_batch(batch_path)

df=df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,plane_1,animal_01/session_01/plane_1.zarr,"{'main': {'var_name_hdf5': 'mov', 'max_shifts'...",{'mean-projection-path': '247620be-d14b-46da-b...,2024-09-09T14:55:09,2024-09-10T09:57:59,42.13 sec,None,247620be-d14b-46da-b6ba-6c73bf607b02
1,cnmf,caiman_phys,247620be-d14b-46da-b6ba-6c73bf607b02/247620be-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': 'bee3b92b-c567-4c7c-b...,2024-09-10T12:50:48,2024-09-10T12:52:31,98.29 sec,None,bee3b92b-c567-4c7c-bb29-d460e08fa980


In [17]:
def read_zarr(path):
    # return zarr.open(path)['mov']
    return da.from_zarr(path, 'mov').compute()

In [4]:
from ipywidgets import Tab, Text, Button, VBox, interact_manual, interactive
@interact_manual(parent_path=str(parent_path), batch_path=str(batch_path))
def start_widget(parent_path, batch_path):
    mc.set_parent_raw_data_path(parent_path)

    df = mc.load_batch(batch_path)

    tab = Tab()

    # mcorr_container = df.mcorr.viz()
    cnmf_container = df.cnmf.viz(start_index=1)

    tab.children = [cnmf_container.show()]
    tab.titles = ["cnmf"]
    display(tab)

interactive(children=(Text(value='/home/mbo/caiman_data_org', continuous_update=False, description='parent_pat…

RFBOutputContext()

RFBOutputContext()

RFBOutputContext()

Draw error
Traceback (most recent call last):
  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/wgpu/gui/base.py", line 24, in log_exception
    yield
  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/wgpu/gui/base.py", line 207, in _draw_frame_and_present
    self.draw_frame()
  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/fastplotlib/layouts/_gridplot.py", line 288, in render
    subplot.render()
  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/fastplotlib/layouts/_plot_area.py", line 281, in render
    self.viewport.render(self.scene, self.camera)
  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/pygfx/utils/viewport.py", line 78, in render
    self.renderer.render(scene, camera, rect=self.rect, flush=flush)
  File "/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/pygfx/renderers/wgpu/_renderer.py", line 480, in render
    container_group = get

...

WrongAlgorithmExtensionError: <cnmf> extension called for a <mcorr> item

WrongAlgorithmExtensionError: <cnmf> extension called for a <mcorr> item

WrongAlgorithmExtensionError: <cnmf> extension called for a <mcorr> item

WrongAlgorithmExtensionError: <cnmf> extension called for a <mcorr> item

WrongAlgorithmExtensionError: <cnmf> extension called for a <mcorr> item